# Explore PUDL FERC Form 2 Data

In [1]:
import sqlite3
import pandas as pd

In [2]:
# Set pandas options
pd.set_option('display.max_columns', None)

## Purpose

The purpose of this notebook will be to explore how to use Pythonb to extract [(FERC)](https://ferc.gov/what-ferc) Form 2 data provided by the Public Utility Data Liberation [(PUDL)](https://catalyst.coop/pudl/) Project.

The goal will be to extract Income Statement data for Montana Gas pipelines from the PUDL Sqlite database and match to the FERC Form 2 exported from the FERC website.

The following natural gas pipelines operate in Montna and file a FERC Form 2:
- Colorado Interstate Gas Company L.L.C.
- Northern Border Pipeline Company
- WBI Energy Transmission, Inc.

## Establish Connection to PUDL Database

The first step will be to connect to the SQLite database downloaded from the PUDL website.

In [3]:
# Connect to the database
conn = sqlite3.connect("ferc2_xbrl.sqlite")

In [4]:
# Create a cursor that lets you execute SQL queries and fetch results
cursor = conn.cursor()

In [5]:
# Get list of table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

## Explore Tables Contained in the Database

The PUDL SQLite database conatins many tables, each representing various schedules and statements. With the database loaded we can now extract a list of the various tables available.

In [6]:
# Assign list of tables to variable
tables = cursor.fetchall()

In [7]:
# Print list of table names
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
corporate_officer_certification_001_duration
identification_001_duration
list_of_schedules_002_duration
general_information_101_duration
control_over_respondent_102_duration
control_over_respondent_102_instant
corporations_controlled_by_respondent_103_duration
corporations_controlled_by_respondent_103_instant
security_holders_and_voting_powers_107_duration
security_holders_and_voting_powers_107_instant
security_holders_and_voting_powers_sequence_107_duration
security_holders_and_voting_powers_sequence_107_instant
important_changes_during_the_quarter_year_108_1_duration
comparative_balance_sheet_assets_and_other_debits_110_instant
comparative_balance_sheet_liabilities_and_other_credits_110_instant
statement_of_income_114_duration
statement_of_accumulated_other_comprehensive_income_and_hedging_activities_117_duration
statement_of_accumulated_other_comprehensive_income_and_hedging_activities_117_instant
retained_earnings_118_duration
retained_earnings_118_instant
r

## Extract Income Statement Data

Below are screenshots of the FERC Form 2 Stateemnt of Income for Colorado Interstate Gas Company:

![](img/ferc-form2-income-statement-01.png)

![](img/ferc-form2-income-statement-02.png)

![](img/ferc-form2-income-statement-03.png)

By looking at the list of tables from earlier, "statement_of_income_114_duration" would appear to be the relevant table name as the FERC Form 2, Statement of Income is on Page 114 of the form. To verify this we can assign this table to a Pandas DataFrame to explore the table columns.

In [8]:
# Assign statement of inocme table to a DataFrame then close the sql connection
df_is = pd.read_sql("SELECT * FROM statement_of_income_114_duration", conn)
conn.close()

In [9]:
df_is.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3011 entries, 0 to 3010
Data columns (total 74 columns):
 #   Column                                                                        Non-Null Count  Dtype  
---  ------                                                                        --------------  -----  
 0   entity_id                                                                     3011 non-null   object 
 1   filing_name                                                                   3011 non-null   object 
 2   publication_time                                                              3011 non-null   object 
 3   start_date                                                                    3011 non-null   object 
 4   end_date                                                                      3011 non-null   object 
 5   utility_type_axis                                                             3011 non-null   object 
 6   revenues_from_merchandising_jobb

In [10]:
# Printout of first 20 entity_id's and filing_name's
df_is.loc[:, ('entity_id', 'filing_name')].head(20)

,entity_id,filing_name
0,C000251,"Trunkline_Gas_Company,_LLC_form2_Q4_1649968474"
1,C000251,"Trunkline_Gas_Company,_LLC_form2_Q4_1649968474"
2,C000251,"Trunkline_Gas_Company,_LLC_form2_Q4_1649968474"
3,C000251,"Trunkline_Gas_Company,_LLC_form2_Q4_1649968474"
4,C000251,"Trunkline_Gas_Company,_LLC_form2_Q4_1649968474"
5,C000251,"Trunkline_Gas_Company,_LLC_form2_Q4_1649968474"
6,C000251,"Trunkline_Gas_Company,_LLC_form2_Q4_1649968474"
7,C000251,"Trunkline_Gas_Company,_LLC_form2_Q4_1649968474"
8,C000251,"Trunkline_Gas_Company,_LLC_form2_Q4_1649968474"
9,C001157,"Cameron_Interstate_Pipeline,_LLC_form2_Q4_1650..."


In [11]:
# Get count of unique entity_id's
unique_entity_count = df_is['entity_id'].nunique()
print(f'Count of unique entities: {unique_entity_count}')

Count of unique entities: 106


The initial DataFrame contains thousands of rows of data for 106 unique filers. For this example, we will start with pulling information for Colorado Interstate Gas Company (CIGC). To be able to do this the DataFrame will need to be filtered. The **entity_id** column contains a unique identifier for each filer and **filing_name** contains the name of the filer. The **filing_name** column can be used to determine which **entity_id** corresponds to CIGC.

In [12]:
# Use str.contains to find CIGC's entity id
df_is[df_is.loc[:, 'filing_name'].str.contains('Colorado_Interstate')]

,entity_id,filing_name,publication_time,start_date,end_date,utility_type_axis,revenues_from_merchandising_jobbing_and_contract_work,losses_from_disposition_of_allowances,donations,maintenance_expense,depreciation_expense_for_asset_retirement_costs,net_other_income_and_deductions,amortization_and_depletion_of_utility_plant,income_taxes_utility_operating_income_other,other_income,amortization_of_premium_on_debt_credit,interest_on_debt_to_associated_companies,net_extraordinary_items,taxes_on_other_income_and_deductions,operating_revenues,investment_tax_credits,net_interest_charges,regulatory_debits,equity_in_earnings_of_subsidiary_companies,provision_for_deferred_income_taxes_other_income_and_deductions,amortization_of_gas_plant_acquisition_adjustments,interest_and_dividend_income,penalties,taxes_other_than_income_taxes_utility_operating_income,regulatory_credits,costs_and_expenses_of_merchandising_jobbing_and_contract_work,other_income_deductions,investment_tax_credit_adjustments,miscellaneous_amortization,income_taxes_extraordinary_items,revenues_from_nonutility_operations,gains_from_disposition_of_plant,allowance_for_borrowed_funds_used_during_construction_credit,investment_tax_credit_adjustments_nonutility_operations,amortization_of_gain_on_reacquired_debt_credit,provision_for_deferred_income_taxes_credit_utility_operating_income,other_interest_expense,interest_on_long_term_debt,income_before_extraordinary_items,extraordinary_items_after_taxes,accretion_expense,life_insurance,allowance_for_other_funds_used_during_construction,amortization_of_loss_on_reacquired_debt,amortization_of_property_losses_unrecovered_plant_and_regulatory_study_costs,extraordinary_income,amortization_of_conversion_expenses,depreciation_expense,expenses_of_nonutility_operations,miscellaneous_nonoperating_income,net_income_loss,loss_on_disposition_of_property,taxes_other_than_income_taxes_other_income_and_deductions,income_taxes_federal,amortization_of_debt_discount_and_expense,expenditures_for_certain_civic_political_and_related_activities,income_taxes_other,income_taxes_utility_operating_income,net_utility_operating_income,provision_for_deferred_income_taxes_credit_other_income_and_deductions,nonoperating_rental_income,extraordinary_deductions,losses_from_disposition_of_utility_plant,operation_expense,utility_operating_expenses,gain_on_disposition_of_property,other_deductions,provisions_for_deferred_income_taxes_utility_operating_income,gains_from_disposition_of_allowances
54,C000981,"Colorado_Interstate_Gas_Company,_L.L.C._form2_...",2022-04-01 02:43:53.000000,2021-01-01,2021-12-31,total,NaN,NaN,NaN,15999678.0,NaN,29977273.0,463860.0,3542106.0,39483478.0,NaN,92114.0,NaN,9440656.0,367801357.0,NaN,22939873.0,15043938.0,35372193.0,1505660.0,NaN,1902638.0,NaN,22888782.0,7722955.0,NaN,65549.0,NaN,NaN,NaN,503157.0,NaN,28268.0,NaN,NaN,1742075.0,94005.0,22412500.0,117163508.0,NaN,NaN,NaN,121604.0,NaN,2662097.0,NaN,NaN,31030845.0,-27354.0,-29167.0,117163508.0,NaN,1329.0,6970530.0,369522.0,11515.0,976073.0,15726866.0,110126108.0,12936.0,NaN,NaN,NaN,150563874.0,257675249.0,1585699.0,54034.0,9218233.0,NaN
55,C000981,"Colorado_Interstate_Gas_Company,_L.L.C._form2_...",2022-04-01 02:43:53.000000,2020-01-01,2020-12-31,total,NaN,NaN,NaN,18372325.0,NaN,28604811.0,503784.0,3429094.0,37845956.0,NaN,120546.0,NaN,9004976.0,377969976.0,NaN,22970774.0,2128237.0,34380175.0,2045843.0,NaN,2764269.0,NaN,20845255.0,2618371.0,NaN,236169.0,NaN,NaN,NaN,387036.0,NaN,32186.0,NaN,NaN,2613124.0,114529.0,22412500.0,130985047.0,NaN,NaN,NaN,143296.0,NaN,2662097.0,NaN,NaN,30698746.0,85396.0,256576.0,130985047.0,49923.0,-421.0,6142325.0,355385.0,7168.0,860101.0,16219451.0,125351010.0,42872.0,NaN,NaN,NaN,152325871.0,252618966.0,NaN,179078.0,10665601.0,NaN
56,C000981,"Colorado_Interstate_Gas_Company,_L.L.C._form2_...",2022-04-01 02:43:53.000000,2021-10-01,2021-10-31,total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30874825.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

Colorado Interstate Gas Company's entity id is **C000981**. We can now use this ID to filter the DataFrame for CIGC.

In [13]:
# Use CIGC's entity id to filter the DataFrame
id_cigc = 'C000981'
df_is_cigc = df_is[df_is['entity_id'] == id_cigc]

In [14]:
df_is_cigc

,entity_id,filing_name,publication_time,start_date,end_date,utility_type_axis,revenues_from_merchandising_jobbing_and_contract_work,losses_from_disposition_of_allowances,donations,maintenance_expense,depreciation_expense_for_asset_retirement_costs,net_other_income_and_deductions,amortization_and_depletion_of_utility_plant,income_taxes_utility_operating_income_other,other_income,amortization_of_premium_on_debt_credit,interest_on_debt_to_associated_companies,net_extraordinary_items,taxes_on_other_income_and_deductions,operating_revenues,investment_tax_credits,net_interest_charges,regulatory_debits,equity_in_earnings_of_subsidiary_companies,provision_for_deferred_income_taxes_other_income_and_deductions,amortization_of_gas_plant_acquisition_adjustments,interest_and_dividend_income,penalties,taxes_other_than_income_taxes_utility_operating_income,regulatory_credits,costs_and_expenses_of_merchandising_jobbing_and_contract_work,other_income_deductions,investment_tax_credit_adjustments,miscellaneous_amortization,income_taxes_extraordinary_items,revenues_from_nonutility_operations,gains_from_disposition_of_plant,allowance_for_borrowed_funds_used_during_construction_credit,investment_tax_credit_adjustments_nonutility_operations,amortization_of_gain_on_reacquired_debt_credit,provision_for_deferred_income_taxes_credit_utility_operating_income,other_interest_expense,interest_on_long_term_debt,income_before_extraordinary_items,extraordinary_items_after_taxes,accretion_expense,life_insurance,allowance_for_other_funds_used_during_construction,amortization_of_loss_on_reacquired_debt,amortization_of_property_losses_unrecovered_plant_and_regulatory_study_costs,extraordinary_income,amortization_of_conversion_expenses,depreciation_expense,expenses_of_nonutility_operations,miscellaneous_nonoperating_income,net_income_loss,loss_on_disposition_of_property,taxes_other_than_income_taxes_other_income_and_deductions,income_taxes_federal,amortization_of_debt_discount_and_expense,expenditures_for_certain_civic_political_and_related_activities,income_taxes_other,income_taxes_utility_operating_income,net_utility_operating_income,provision_for_deferred_income_taxes_credit_other_income_and_deductions,nonoperating_rental_income,extraordinary_deductions,losses_from_disposition_of_utility_plant,operation_expense,utility_operating_expenses,gain_on_disposition_of_property,other_deductions,provisions_for_deferred_income_taxes_utility_operating_income,gains_from_disposition_of_allowances
54,C000981,"Colorado_Interstate_Gas_Company,_L.L.C._form2_...",2022-04-01 02:43:53.000000,2021-01-01,2021-12-31,total,NaN,NaN,NaN,15999678.0,NaN,29977273.0,463860.0,3542106.0,39483478.0,NaN,92114.0,NaN,9440656.0,367801357.0,NaN,22939873.0,15043938.0,35372193.0,1505660.0,NaN,1902638.0,NaN,22888782.0,7722955.0,NaN,65549.0,NaN,NaN,NaN,503157.0,NaN,28268.0,NaN,NaN,1742075.0,94005.0,22412500.0,117163508.0,NaN,NaN,NaN,121604.0,NaN,2662097.0,NaN,NaN,31030845.0,-27354.0,-29167.0,117163508.0,NaN,1329.0,6970530.0,369522.0,11515.0,976073.0,15726866.0,110126108.0,12936.0,NaN,NaN,NaN,150563874.0,257675249.0,1585699.0,54034.0,9218233.0,NaN
55,C000981,"Colorado_Interstate_Gas_Company,_L.L.C._form2_...",2022-04-01 02:43:53.000000,2020-01-01,2020-12-31,total,NaN,NaN,NaN,18372325.0,NaN,28604811.0,503784.0,3429094.0,37845956.0,NaN,120546.0,NaN,9004976.0,377969976.0,NaN,22970774.0,2128237.0,34380175.0,2045843.0,NaN,2764269.0,NaN,20845255.0,2618371.0,NaN,236169.0,NaN,NaN,NaN,387036.0,NaN,32186.0,NaN,NaN,2613124.0,114529.0,22412500.0,130985047.0,NaN,NaN,NaN,143296.0,NaN,2662097.0,NaN,NaN,30698746.0,85396.0,256576.0,130985047.0,49923.0,-421.0,6142325.0,355385.0,7168.0,860101.0,16219451.0,125351010.0,42872.0,NaN,NaN,NaN,152325871.0,252618966.0,NaN,179078.0,10665601.0,NaN
56,C000981,"Colorado_Interstate_Gas_Company,_L.L.C._form2_...",2022-04-01 02:43:53.000000,2021-10-01,2021-10-31,total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30874825.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

From reviewing the above there will be overlap of data due to the fact that for each FERC form there is current and prior year data reported. This results in prior year data being duplicated. For this section of the analyis our goal is to match the current year FERC Form 2, so we will further filter the data to the most recent form. To do this we can filter using the **publication_time** column and only keep rows published in 2025.

In [15]:
# Filter the DataFrame by publication_time
df_is_cigc_25 = df_is_cigc[df_is_cigc.loc[:, 'publication_time'].str.contains('2025-')]

In [16]:
df_is_cigc_25

,entity_id,filing_name,publication_time,start_date,end_date,utility_type_axis,revenues_from_merchandising_jobbing_and_contract_work,losses_from_disposition_of_allowances,donations,maintenance_expense,depreciation_expense_for_asset_retirement_costs,net_other_income_and_deductions,amortization_and_depletion_of_utility_plant,income_taxes_utility_operating_income_other,other_income,amortization_of_premium_on_debt_credit,interest_on_debt_to_associated_companies,net_extraordinary_items,taxes_on_other_income_and_deductions,operating_revenues,investment_tax_credits,net_interest_charges,regulatory_debits,equity_in_earnings_of_subsidiary_companies,provision_for_deferred_income_taxes_other_income_and_deductions,amortization_of_gas_plant_acquisition_adjustments,interest_and_dividend_income,penalties,taxes_other_than_income_taxes_utility_operating_income,regulatory_credits,costs_and_expenses_of_merchandising_jobbing_and_contract_work,other_income_deductions,investment_tax_credit_adjustments,miscellaneous_amortization,income_taxes_extraordinary_items,revenues_from_nonutility_operations,gains_from_disposition_of_plant,allowance_for_borrowed_funds_used_during_construction_credit,investment_tax_credit_adjustments_nonutility_operations,amortization_of_gain_on_reacquired_debt_credit,provision_for_deferred_income_taxes_credit_utility_operating_income,other_interest_expense,interest_on_long_term_debt,income_before_extraordinary_items,extraordinary_items_after_taxes,accretion_expense,life_insurance,allowance_for_other_funds_used_during_construction,amortization_of_loss_on_reacquired_debt,amortization_of_property_losses_unrecovered_plant_and_regulatory_study_costs,extraordinary_income,amortization_of_conversion_expenses,depreciation_expense,expenses_of_nonutility_operations,miscellaneous_nonoperating_income,net_income_loss,loss_on_disposition_of_property,taxes_other_than_income_taxes_other_income_and_deductions,income_taxes_federal,amortization_of_debt_discount_and_expense,expenditures_for_certain_civic_political_and_related_activities,income_taxes_other,income_taxes_utility_operating_income,net_utility_operating_income,provision_for_deferred_income_taxes_credit_other_income_and_deductions,nonoperating_rental_income,extraordinary_deductions,losses_from_disposition_of_utility_plant,operation_expense,utility_operating_expenses,gain_on_disposition_of_property,other_deductions,provisions_for_deferred_income_taxes_utility_operating_income,gains_from_disposition_of_allowances
2814,C000981,"Colorado_Interstate_Gas_Company,_L.L.C._form2_...",2025-03-27 22:02:24.000000,2024-01-01,2024-12-31,total,NaN,NaN,NaN,32979655.0,NaN,36381297.0,165213.0,1747527.0,47620477.0,NaN,620753.0,NaN,11177652.0,332110238.0,NaN,23273522.0,5460424.0,28848832.0,643213.0,NaN,15456974.0,NaN,18340897.0,7532309.0,NaN,61528.0,NaN,NaN,NaN,553247.0,NaN,409845.0,NaN,NaN,1216197.0,234425.0,22412500.0,101787910.0,NaN,NaN,NaN,1706580.0,NaN,NaN,NaN,NaN,32360350.0,97180.0,1152024.0,101787910.0,4139.0,-8110.0,9429387.0,415689.0,74298.0,1113162.0,8674348.0,88680135.0,NaN,NaN,NaN,NaN,143456709.0,243430103.0,NaN,-16909.0,8993486.0,NaN
2815,C000981,"Colorado_Interstate_Gas_Company,_L.L.C._form2_...",2025-03-27 22:02:24.000000,2024-01-01,2024-12-31,ferc:GasUtilityMember,NaN,NaN,NaN,32979655.0,NaN,NaN,165213.0,1747527.0,NaN,NaN,NaN,NaN,NaN,332110238.0,NaN,NaN,5460424.0,NaN,NaN,NaN,NaN,NaN,18340897.0,7532309.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1216197.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32360350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8674348.0,88680135.0,NaN,NaN,NaN,NaN,143456709.0,243430103.0,NaN,NaN,8993486.0,NaN
2816,C000981,"Colorado_Interstate_Gas_Company,_L.L.C._form2_...",2025-03-27 22:02:24.000000,2023-01-01,2023-12-31,ferc:GasUtilityMember,NaN,NaN,NaN,28854725.0,NaN,NaN,224340.0,1871571.0,NaN,NaN,NaN,NaN,NaN,332489669.0,NaN,NaN,5006164.0,NaN,NaN,NaN,NaN,NaN,18979324.0,3259665.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1869427.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31609006

From a quick review this does appear to be the correct information. However, the column orders do not match to the FERC Form 2. To be able to complete a full comparison of the data the columns are going to need to be rearanged in an order that matches to the Form 2. It's also worth noting that there are rows for current and prior year totals as well as current and prior year rows for Elec. Utility, Gas Utility, and Other Utility.

In [17]:
# Get a list of sorted column names
column_names = df_is_cigc_25.columns
column_names = column_names.sort_values()

In [18]:
# Print sorted list
for column in column_names:
    print(column)

accretion_expense
allowance_for_borrowed_funds_used_during_construction_credit
allowance_for_other_funds_used_during_construction
amortization_and_depletion_of_utility_plant
amortization_of_conversion_expenses
amortization_of_debt_discount_and_expense
amortization_of_gain_on_reacquired_debt_credit
amortization_of_gas_plant_acquisition_adjustments
amortization_of_loss_on_reacquired_debt
amortization_of_premium_on_debt_credit
amortization_of_property_losses_unrecovered_plant_and_regulatory_study_costs
costs_and_expenses_of_merchandising_jobbing_and_contract_work
depreciation_expense
depreciation_expense_for_asset_retirement_costs
donations
end_date
entity_id
equity_in_earnings_of_subsidiary_companies
expenditures_for_certain_civic_political_and_related_activities
expenses_of_nonutility_operations
extraordinary_deductions
extraordinary_income
extraordinary_items_after_taxes
filing_name
gain_on_disposition_of_property
gains_from_disposition_of_allowances
gains_from_disposition_of_plant
inc

This sorted list was then used to compare to the XBRL form 2 available from the FERC website for Coloraod Interstate Gas Company. The columns match to a specific xbrl tag that correlates with an item in the statements. For example the below screenshot shows the xbrl tag for the first item in the statement, operating revenues:

![](img/ferc-form2-xbrl-tag.png)

The XBRL tag "OperatingReveneus" correlates with column "operating_revenues" in the PUDL database. \
\
By looking up the tag and matching to the list of column names, a list was put together that allowed the database to be sorted in the same order as the Statement of Income.

In [19]:
# Produce a printout of the DataFrame in an order that matches to the Income Statement
df_is_cigc_25.loc[:, [
    'entity_id',
    'filing_name',
    'publication_time',
    'utility_type_axis',
    'start_date',
    'end_date',
    'operating_revenues',
    'operation_expense',
    'maintenance_expense',
    'depreciation_expense',
    'depreciation_expense_for_asset_retirement_costs',
    'amortization_and_depletion_of_utility_plant',
    'amortization_of_gas_plant_acquisition_adjustments',
    'amortization_of_property_losses_unrecovered_plant_and_regulatory_study_costs',
    'amortization_of_conversion_expenses',
    'regulatory_debits',
    'regulatory_credits',
    'taxes_other_than_income_taxes_utility_operating_income',
    'income_taxes_utility_operating_income',
    'income_taxes_utility_operating_income_other',
    'provisions_for_deferred_income_taxes_utility_operating_income',
    'provision_for_deferred_income_taxes_credit_utility_operating_income',
    'investment_tax_credit_adjustments',
    'gains_from_disposition_of_plant',
    'losses_from_disposition_of_utility_plant',
    'gains_from_disposition_of_allowances',
    'losses_from_disposition_of_allowances',
    'accretion_expense',
    'utility_operating_expenses',
    'net_utility_operating_income',
    'revenues_from_merchandising_jobbing_and_contract_work',
    'costs_and_expenses_of_merchandising_jobbing_and_contract_work',
    'revenues_from_nonutility_operations',
    'expenses_of_nonutility_operations',
    'nonoperating_rental_income',
    'equity_in_earnings_of_subsidiary_companies',
    'interest_and_dividend_income',
    'allowance_for_other_funds_used_during_construction',
    'miscellaneous_nonoperating_income',
    'gain_on_disposition_of_property',
    'other_income',
    'loss_on_disposition_of_property',
    'miscellaneous_amortization',
    'donations',
    'life_insurance',
    'penalties',
    'expenditures_for_certain_civic_political_and_related_activities',
    'other_deductions',
    'other_income_deductions',
    'taxes_other_than_income_taxes_other_income_and_deductions',
    'income_taxes_federal',
    'income_taxes_other',
    'provision_for_deferred_income_taxes_other_income_and_deductions',
    'provision_for_deferred_income_taxes_credit_other_income_and_deductions',
    'investment_tax_credit_adjustments_nonutility_operations',
    'investment_tax_credits',
    'taxes_on_other_income_and_deductions',
    'net_other_income_and_deductions',
    'interest_on_long_term_debt',
    'amortization_of_debt_discount_and_expense',
    'amortization_of_loss_on_reacquired_debt',
    'amortization_of_premium_on_debt_credit',
    'amortization_of_gain_on_reacquired_debt_credit',
    'interest_on_debt_to_associated_companies',
    'other_interest_expense',
    'allowance_for_borrowed_funds_used_during_construction_credit',
    'net_interest_charges',
    'income_before_extraordinary_items',
    'extraordinary_income',
    'extraordinary_deductions',
    'net_extraordinary_items',
    'income_taxes_extraordinary_items',
    'extraordinary_items_after_taxes',
    'net_income_loss'
    ]]

,entity_id,filing_name,publication_time,utility_type_axis,start_date,end_date,operating_revenues,operation_expense,maintenance_expense,depreciation_expense,depreciation_expense_for_asset_retirement_costs,amortization_and_depletion_of_utility_plant,amortization_of_gas_plant_acquisition_adjustments,amortization_of_property_losses_unrecovered_plant_and_regulatory_study_costs,amortization_of_conversion_expenses,regulatory_debits,regulatory_credits,taxes_other_than_income_taxes_utility_operating_income,income_taxes_utility_operating_income,income_taxes_utility_operating_income_other,provisions_for_deferred_income_taxes_utility_operating_income,provision_for_deferred_income_taxes_credit_utility_operating_income,investment_tax_credit_adjustments,gains_from_disposition_of_plant,losses_from_disposition_of_utility_plant,gains_from_disposition_of_allowances,losses_from_disposition_of_allowances,accretion_expense,utility_operating_expenses,net_utility_operating_income,revenues_from_merchandising_jobbing_and_contract_work,costs_and_expenses_of_merchandising_jobbing_and_contract_work,revenues_from_nonutility_operations,expenses_of_nonutility_operations,nonoperating_rental_income,equity_in_earnings_of_subsidiary_companies,interest_and_dividend_income,allowance_for_other_funds_used_during_construction,miscellaneous_nonoperating_income,gain_on_disposition_of_property,other_income,loss_on_disposition_of_property,miscellaneous_amortization,donations,life_insurance,penalties,expenditures_for_certain_civic_political_and_related_activities,other_deductions,other_income_deductions,taxes_other_than_income_taxes_other_income_and_deductions,income_taxes_federal,income_taxes_other,provision_for_deferred_income_taxes_other_income_and_deductions,provision_for_deferred_income_taxes_credit_other_income_and_deductions,investment_tax_credit_adjustments_nonutility_operations,investment_tax_credits,taxes_on_other_income_and_deductions,net_other_income_and_deductions,interest_on_long_term_debt,amortization_of_debt_discount_and_expense,amortization_of_loss_on_reacquired_debt,amortization_of_premium_on_debt_credit,amortization_of_gain_on_reacquired_debt_credit,interest_on_debt_to_associated_companies,other_interest_expense,allowance_for_borrowed_funds_used_during_construction_credit,net_interest_charges,income_before_extraordinary_items,extraordinary_income,extraordinary_deductions,net_extraordinary_items,income_taxes_extraordinary_items,extraordinary_items_after_taxes,net_income_loss
2814,C000981,"Colorado_Interstate_Gas_Company,_L.L.C._form2_...",2025-03-27 22:02:24.000000,total,2024-01-01,2024-12-31,332110238.0,143456709.0,32979655.0,32360350.0,NaN,165213.0,NaN,NaN,NaN,5460424.0,7532309.0,18340897.0,8674348.0,1747527.0,8993486.0,1216197.0,NaN,NaN,NaN,NaN,NaN,NaN,243430103.0,88680135.0,NaN,NaN,553247.0,97180.0,NaN,28848832.0,15456974.0,1706580.0,1152024.0,NaN,47620477.0,4139.0,NaN,NaN,NaN,NaN,74298.0,-16909.0,61528.0,-8110.0,9429387.0,1113162.0,643213.0,NaN,NaN,NaN,11177652.0,36381297.0,22412500.0,415689.0,NaN,NaN,NaN,620753.0,234425.0,409845.0,23273522.0,101787910.0,NaN,NaN,NaN,NaN,NaN,101787910.0
2815,C000981,"Colorado_Interstate_Gas_Company,_L.L.C._form2_...",2025-03-27 22:02:24.000000,ferc:GasUtilityMember,2024-01-01,2024-12-31,332110238.0,143456709.0,32979655.0,32360350.0,NaN,165213.0,NaN,NaN,NaN,5460424.0,7532309.0,18340897.0,8674348.0,1747527.0,8993486.0,1216197.0,NaN,NaN,NaN,NaN,NaN,NaN,243430103.0,88680135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2816,C000981,"Colorado_Interstate_Gas_Company,_L.L.C._form2_...",2025-03-27 22:02:24.000000,ferc:GasUtilityMember,2023-01-01,2023-12-31,332489669.0,143854809.0,28854725.0,31609006.0,NaN,224340.0,NaN,NaN,NaN,5006164.0,3259665.0,18979324.0,9310819.0,1871571.0,8917791.0,1869427.0,NaN,NaN,NaN,NaN,NaN,NaN,243499457.0,88990212.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

Comparing the sorted DataFrame to the FERC Form 2 for Colorado Interstate Gas does materially match.

## Income Statement Extract Function

With the correct SQL table and the order of columns identified a function can be developed to extract the Statement of Income for each entity id.

In [20]:
# Define the Income Statement function
def get_ferc2_income_statement(db_file, subject_id):

    # Connect to the sql database using sqlite3
    conn = sqlite3.connect(db_file)

    # Query the database file and assign to pandas dataframe
    sql_table = 'statement_of_income_114_duration'
    sql_query = f'SELECT * FROM {sql_table}'
    df = pd.read_sql(sql_query + " WHERE entity_id = ?", conn, params=(subject_id,))
    
    # Close the database connection
    conn.close()

    # Rearange columns to match income statement format
    income_statement_items = (
        'entity_id',
        'filing_name',
        'publication_time',
        'utility_type_axis',
        'start_date',
        'end_date',
        'operating_revenues',
        'operation_expense',
        'maintenance_expense',
        'depreciation_expense',
        'depreciation_expense_for_asset_retirement_costs',
        'amortization_and_depletion_of_utility_plant',
        'amortization_of_gas_plant_acquisition_adjustments',
        'amortization_of_property_losses_unrecovered_plant_and_regulatory_study_costs',
        'amortization_of_conversion_expenses',
        'regulatory_debits',
        'regulatory_credits',
        'taxes_other_than_income_taxes_utility_operating_income',
        'income_taxes_utility_operating_income',
        'income_taxes_utility_operating_income_other',
        'provisions_for_deferred_income_taxes_utility_operating_income',
        'provision_for_deferred_income_taxes_credit_utility_operating_income',
        'investment_tax_credit_adjustments',
        'gains_from_disposition_of_plant',
        'losses_from_disposition_of_utility_plant',
        'gains_from_disposition_of_allowances',
        'losses_from_disposition_of_allowances',
        'accretion_expense',
        'utility_operating_expenses',
        'net_utility_operating_income',
        'revenues_from_merchandising_jobbing_and_contract_work',
        'costs_and_expenses_of_merchandising_jobbing_and_contract_work',
        'revenues_from_nonutility_operations',
        'expenses_of_nonutility_operations',
        'nonoperating_rental_income',
        'equity_in_earnings_of_subsidiary_companies',
        'interest_and_dividend_income',
        'allowance_for_other_funds_used_during_construction',
        'miscellaneous_nonoperating_income',
        'gain_on_disposition_of_property',
        'other_income',
        'loss_on_disposition_of_property',
        'miscellaneous_amortization',
        'donations',
        'life_insurance',
        'penalties',
        'expenditures_for_certain_civic_political_and_related_activities',
        'other_deductions',
        'other_income_deductions',
        'taxes_other_than_income_taxes_other_income_and_deductions',
        'income_taxes_federal',
        'income_taxes_other',
        'provision_for_deferred_income_taxes_other_income_and_deductions',
        'provision_for_deferred_income_taxes_credit_other_income_and_deductions',
        'investment_tax_credit_adjustments_nonutility_operations',
        'investment_tax_credits',
        'taxes_on_other_income_and_deductions',
        'net_other_income_and_deductions',
        'interest_on_long_term_debt',
        'amortization_of_debt_discount_and_expense',
        'amortization_of_loss_on_reacquired_debt',
        'amortization_of_premium_on_debt_credit',
        'amortization_of_gain_on_reacquired_debt_credit',
        'interest_on_debt_to_associated_companies',
        'other_interest_expense',
        'allowance_for_borrowed_funds_used_during_construction_credit',
        'net_interest_charges',
        'income_before_extraordinary_items',
        'extraordinary_income',
        'extraordinary_deductions',
        'net_extraordinary_items',
        'income_taxes_extraordinary_items',
        'extraordinary_items_after_taxes',
        'net_income_loss'
    )
    
    df = df.loc[:, income_statement_items]
    
    return df

We now can test the function using Colorado Interstate Gas entity id "C000981".

In [21]:
# Set function parameters
db_file = 'ferc2_xbrl.sqlite'
subject_id = 'C000981'

In [22]:
# Query the PUDL SQL Database using the function
df_cigc = get_ferc2_income_statement(db_file, subject_id)

In [23]:
df_cigc

,entity_id,filing_name,publication_time,utility_type_axis,start_date,end_date,operating_revenues,operation_expense,maintenance_expense,depreciation_expense,depreciation_expense_for_asset_retirement_costs,amortization_and_depletion_of_utility_plant,amortization_of_gas_plant_acquisition_adjustments,amortization_of_property_losses_unrecovered_plant_and_regulatory_study_costs,amortization_of_conversion_expenses,regulatory_debits,regulatory_credits,taxes_other_than_income_taxes_utility_operating_income,income_taxes_utility_operating_income,income_taxes_utility_operating_income_other,provisions_for_deferred_income_taxes_utility_operating_income,provision_for_deferred_income_taxes_credit_utility_operating_income,investment_tax_credit_adjustments,gains_from_disposition_of_plant,losses_from_disposition_of_utility_plant,gains_from_disposition_of_allowances,losses_from_disposition_of_allowances,accretion_expense,utility_operating_expenses,net_utility_operating_income,revenues_from_merchandising_jobbing_and_contract_work,costs_and_expenses_of_merchandising_jobbing_and_contract_work,revenues_from_nonutility_operations,expenses_of_nonutility_operations,nonoperating_rental_income,equity_in_earnings_of_subsidiary_companies,interest_and_dividend_income,allowance_for_other_funds_used_during_construction,miscellaneous_nonoperating_income,gain_on_disposition_of_property,other_income,loss_on_disposition_of_property,miscellaneous_amortization,donations,life_insurance,penalties,expenditures_for_certain_civic_political_and_related_activities,other_deductions,other_income_deductions,taxes_other_than_income_taxes_other_income_and_deductions,income_taxes_federal,income_taxes_other,provision_for_deferred_income_taxes_other_income_and_deductions,provision_for_deferred_income_taxes_credit_other_income_and_deductions,investment_tax_credit_adjustments_nonutility_operations,investment_tax_credits,taxes_on_other_income_and_deductions,net_other_income_and_deductions,interest_on_long_term_debt,amortization_of_debt_discount_and_expense,amortization_of_loss_on_reacquired_debt,amortization_of_premium_on_debt_credit,amortization_of_gain_on_reacquired_debt_credit,interest_on_debt_to_associated_companies,other_interest_expense,allowance_for_borrowed_funds_used_during_construction_credit,net_interest_charges,income_before_extraordinary_items,extraordinary_income,extraordinary_deductions,net_extraordinary_items,income_taxes_extraordinary_items,extraordinary_items_after_taxes,net_income_loss
0,C000981,"Colorado_Interstate_Gas_Company,_L.L.C._form2_...",2022-04-01 02:43:53.000000,total,2021-01-01,2021-12-31,367801357.0,150563874.0,15999678.0,31030845.0,None,463860.0,None,2662097.0,None,15043938.0,7722955.0,22888782.0,15726866.0,3542106.0,9218233.0,1742075.0,None,None,None,None,None,None,257675249.0,110126108.0,None,None,503157.0,-27354.0,None,35372193.0,1902638.0,121604.0,-29167.0,1585699.0,39483478.0,NaN,None,None,None,NaN,11515.0,54034.0,65549.0,1329.0,6970530.0,976073.0,1505660.0,12936.0,None,None,9440656.0,29977273.0,22412500.0,369522.0,None,None,None,92114.0,94005.0,28268.0,22939873.0,117163508.0,None,None,None,None,None,117163508.0
1,C000981,"Colorado_Interstate_Gas_Company,_L.L.C._form2_...",2022-04-01 02:43:53.000000,total,2020-01-01,2020-12-31,377969976.0,152325871.0,18372325.0,30698746.0,None,503784.0,None,2662097.0,None,2128237.0,2618371.0,20845255.0,16219451.0,3429094.0,10665601.0,2613124.0,None,None,None,None,None,None,252618966.0,125351010.0,None,None,387036.0,85396.0,None,34380175.0,2764269.0,143296.0,256576.0,NaN,37845956.0,49923.0,None,None,None,NaN,7168.0,179078.0,236169.0,-421.0,6142325.0,860101.0,2045843.0,42872.0,None,None,9004976.0,28604811.0,22412500.0,355385.0,None,None,None,120546.0,114529.0,32186.0,22970774.0,130985047.0,None,None,None,None,None,130985047.0
2,C000981,"Colorado_Interstate_Gas_Company,_L.L.C._form2_...",2022-04-01 02:43:53.000000,total,2021-10-01,2021-10-31,30874825.0,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

The functions appears to work as the resulting DataFrame matches to the FERC Form 2 Statement of Income for CIGC. It now can be tested with additional entities.

## Extract Income Statement for Other Entities

This function can now be tested with the remaining Montana gas pipelines identified at the beginning. \
The sql database was reviewed to determine the entity_id's that match to the remaining Montana gas pipelines:
- Northern Border Pipeline Company = Entity ID: C000626 
- WBI Energy Transmission Inc = Entity ID: C000111

In [24]:
# Assign additional entity_id's to variables
id_northern_border_pipeline = 'C000626'
id_wbi_energy_transmission = 'C000111'

In [25]:
# Extract additional income statements using entitiy_id's
df_northern_border_pipeline = get_ferc2_income_statement(db_file, id_northern_border_pipeline)
df_wbi_energy_transmission = get_ferc2_income_statement(db_file, id_wbi_energy_transmission)

In [26]:
df_northern_border_pipeline

,entity_id,filing_name,publication_time,utility_type_axis,start_date,end_date,operating_revenues,operation_expense,maintenance_expense,depreciation_expense,depreciation_expense_for_asset_retirement_costs,amortization_and_depletion_of_utility_plant,amortization_of_gas_plant_acquisition_adjustments,amortization_of_property_losses_unrecovered_plant_and_regulatory_study_costs,amortization_of_conversion_expenses,regulatory_debits,regulatory_credits,taxes_other_than_income_taxes_utility_operating_income,income_taxes_utility_operating_income,income_taxes_utility_operating_income_other,provisions_for_deferred_income_taxes_utility_operating_income,provision_for_deferred_income_taxes_credit_utility_operating_income,investment_tax_credit_adjustments,gains_from_disposition_of_plant,losses_from_disposition_of_utility_plant,gains_from_disposition_of_allowances,losses_from_disposition_of_allowances,accretion_expense,utility_operating_expenses,net_utility_operating_income,revenues_from_merchandising_jobbing_and_contract_work,costs_and_expenses_of_merchandising_jobbing_and_contract_work,revenues_from_nonutility_operations,expenses_of_nonutility_operations,nonoperating_rental_income,equity_in_earnings_of_subsidiary_companies,interest_and_dividend_income,allowance_for_other_funds_used_during_construction,miscellaneous_nonoperating_income,gain_on_disposition_of_property,other_income,loss_on_disposition_of_property,miscellaneous_amortization,donations,life_insurance,penalties,expenditures_for_certain_civic_political_and_related_activities,other_deductions,other_income_deductions,taxes_other_than_income_taxes_other_income_and_deductions,income_taxes_federal,income_taxes_other,provision_for_deferred_income_taxes_other_income_and_deductions,provision_for_deferred_income_taxes_credit_other_income_and_deductions,investment_tax_credit_adjustments_nonutility_operations,investment_tax_credits,taxes_on_other_income_and_deductions,net_other_income_and_deductions,interest_on_long_term_debt,amortization_of_debt_discount_and_expense,amortization_of_loss_on_reacquired_debt,amortization_of_premium_on_debt_credit,amortization_of_gain_on_reacquired_debt_credit,interest_on_debt_to_associated_companies,other_interest_expense,allowance_for_borrowed_funds_used_during_construction_credit,net_interest_charges,income_before_extraordinary_items,extraordinary_income,extraordinary_deductions,net_extraordinary_items,income_taxes_extraordinary_items,extraordinary_items_after_taxes,net_income_loss
0,C000626,Northern_Border_Pipeline_Company_form2_Q4_1649...,2022-04-06 21:06:10.000000,ferc:GasUtilityMember,2020-01-01,2020-12-31,307802925.0,49732609.0,4868187.0,60744665.0,None,553596.0,None,None,None,1795074.0,1406722.0,23097886.0,NaN,NaN,NaN,NaN,None,None,None,None,None,None,139385295.0,168417630.0,None,None,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,NaN,NaN,None,None,None,NaN,None,NaN
1,C000626,Northern_Border_Pipeline_Company_form2_Q4_1649...,2022-04-06 21:06:10.000000,total,2021-11-01,2021-11-30,23552352.0,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None,None,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,NaN,NaN,None,None,None,NaN,None,NaN
2,C000626,Northern_Border_Pipeline_Company_form2_Q4_1649...,2022-04-06 21:06:10.000000,total,2021-01-01,2021-12-31,279657286.0,48728791.0,4643830.0,61936425.0,None,954589.0,None,None,None,1756820.0,1057177.0,22450730.0,22173621.0,5084348.0,682103.0,4731530.0,None,None,None,None,None,None,162622550.0,117034736.0,None,None,3237426.0,None,None,None,47876.0,1039946.0,1943920.0,NaN,6269168.0,None,30849.0,NaN,None,4833.0,31932.0,84930.0,152544.0,None,-1071696.0,-251685.0,2443501.0,NaN,None,None,1120120.0,4996504.0,17201851.0,379527.0,None,None,None,None,14253.0,175075.0,17420556.0,10461

In [27]:
df_wbi_energy_transmission

,entity_id,filing_name,publication_time,utility_type_axis,start_date,end_date,operating_revenues,operation_expense,maintenance_expense,depreciation_expense,depreciation_expense_for_asset_retirement_costs,amortization_and_depletion_of_utility_plant,amortization_of_gas_plant_acquisition_adjustments,amortization_of_property_losses_unrecovered_plant_and_regulatory_study_costs,amortization_of_conversion_expenses,regulatory_debits,regulatory_credits,taxes_other_than_income_taxes_utility_operating_income,income_taxes_utility_operating_income,income_taxes_utility_operating_income_other,provisions_for_deferred_income_taxes_utility_operating_income,provision_for_deferred_income_taxes_credit_utility_operating_income,investment_tax_credit_adjustments,gains_from_disposition_of_plant,losses_from_disposition_of_utility_plant,gains_from_disposition_of_allowances,losses_from_disposition_of_allowances,accretion_expense,utility_operating_expenses,net_utility_operating_income,revenues_from_merchandising_jobbing_and_contract_work,costs_and_expenses_of_merchandising_jobbing_and_contract_work,revenues_from_nonutility_operations,expenses_of_nonutility_operations,nonoperating_rental_income,equity_in_earnings_of_subsidiary_companies,interest_and_dividend_income,allowance_for_other_funds_used_during_construction,miscellaneous_nonoperating_income,gain_on_disposition_of_property,other_income,loss_on_disposition_of_property,miscellaneous_amortization,donations,life_insurance,penalties,expenditures_for_certain_civic_political_and_related_activities,other_deductions,other_income_deductions,taxes_other_than_income_taxes_other_income_and_deductions,income_taxes_federal,income_taxes_other,provision_for_deferred_income_taxes_other_income_and_deductions,provision_for_deferred_income_taxes_credit_other_income_and_deductions,investment_tax_credit_adjustments_nonutility_operations,investment_tax_credits,taxes_on_other_income_and_deductions,net_other_income_and_deductions,interest_on_long_term_debt,amortization_of_debt_discount_and_expense,amortization_of_loss_on_reacquired_debt,amortization_of_premium_on_debt_credit,amortization_of_gain_on_reacquired_debt_credit,interest_on_debt_to_associated_companies,other_interest_expense,allowance_for_borrowed_funds_used_during_construction_credit,net_interest_charges,income_before_extraordinary_items,extraordinary_income,extraordinary_deductions,net_extraordinary_items,income_taxes_extraordinary_items,extraordinary_items_after_taxes,net_income_loss
0,C000111,"WBI_Energy_Transmission,_Inc._form2_Q4_1650061854",2022-04-15 22:30:54.000000,total,2021-12-01,2021-12-31,11154129.0,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,NaN
1,C000111,"WBI_Energy_Transmission,_Inc._form2_Q4_1650061854",2022-04-15 22:30:54.000000,total,2020-01-01,2020-12-31,128127689.0,36524589.0,7966557.0,19948862.0,184180.0,465952.0,None,None,None,3336625.0,6624009.0,12176945.0,7613750.0,1794263.0,10561474.0,13395166.0,None,None,240.0,None,None,4326027.0,84880289.0,43247400.0,None,None,None,None,115805.0,None,18521.0,812998.0,35119.0,969214.0,1951657.0,653522.0,None,211136.0,None,NaN,47445.0,575967.0,1488070.0,948.0,526838.0,123738.0,None,NaN,None,None,651524.0,-187937.0,7503920.0,86726.0,NaN,None,None,17253.0,286445.0,287497.0,7606847.0,35452616.0,None,None,None,None,None,35452616.0
2,C000111,"WBI_Energy_Transmission,_Inc._form2_Q4_1650061854",2022-04-15 22:30:54.000000,total,2021-01-01,2021-12-31,128976601.0,38370864.0,8477820.0,20031568.0,110817.0,510525.0,None,None,None,2693407.0,5859175.0,12491275.0,3573141.0,733698.0,15165325.0,11858394.0,None,None,92216.0,None,None,1574678.0,86107765.0,42868836.0,None,None,None,None,10000.0,None,14657.0,6275295.0,40006.0,762095.0,7102053.0,9673.0,None,264129.0,None,NaN,47949.0,893456.

## Conclusion

We were able to successfully extract a complete FERC Form 2 Statement of Income from the PUDL SQL database and were able to build a custom extract function. This function can now be used to extract Form 2 Income Statement data as needed.